In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
file = '/content/drive/MyDrive/Colab Notebooks/HSDC Time series/Time_series_analysis_and_forecast_DATASET.csv'
electricity_file = pd.read_csv(file)

In [ ]:
electricity_file.head()

In [ ]:
electricity_file.shape

In [ ]:
electricity_file.isnull().values.any()

In [ ]:
electricity_file["FullDate"] = pd.to_datetime(electricity_file["FullDate"])
electricity_file.set_index("FullDate", inplace=True)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(electricity_file.index, electricity_file.ElecPrice, marker='*')
plt.grid()
plt.xlabel('Year')
plt.ylabel('ElecPrice')

In [ ]:

#Resample data to a daily frequency and sum
electricity_file = electricity_file.resample('D').sum()

In [ ]:
electricity_file

In [ ]:
train = electricity_file[0:2757]
test = electricity_file[2757:]

In [ ]:
train.shape

In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
train = train.rename(columns={"FullDate": "ds", "ElecPrice": "y"})
train.head()

In [ ]:
def SysLoad(ds):
  date = pd.to_datetime(ds)
  if date.weekday() == 6 and (date.month > 8 or date.month < 2):
    return 1
  else:
    return 0

def Tmax(ds):
  date = pd.to_datetime(ds)
  if date.weekday() == 6 and (date.month > 8 or date.month < 2):
    return 1
  else:
    return 0

def GasPrice(ds):
  date = pd.to_datetime(ds)
  if date.weekday() == 6 and (date.month > 8 or date.month < 2):
    return 1
  else:
    return 0
train['add1'] = train['ds'].apply(SysLoad)
train['add2'] = train['ds'].apply(Tmax)
train['add3'] = train['ds'].apply(GasPrice)

model = Prophet()
model.add_regressor('SysLoad')
                    
model.add_regressor('Tmax')
                    
model.add_regressor('GasPrice')

model.fit(train)

future = model.make_future_dataframe(periods= 20 , freq= 'M' )
future['SysLoad'] = future['ds'].apply(SysLoad)
future['Tmax'] = future['ds'].apply(Tmax)
future['GasPrice'] = future['ds'].apply(GasPrice)

forecast = model.predict(future)
fig = model.plot_components(forecast)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def rmse(predictions, targets):
    differences = predictions - targets
    differences_squared = differences ** 2
    mean_of_differences_squared = np.mean(differences_squared)
    rmse_val = np.sqrt(mean_of_differences_squared)
    return rmse_val

    